In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import pandas as pd
import tensorflow as tf


In [2]:
tf.logging.set_verbosity(tf.logging.INFO)

In [3]:
COLUMNS = ["crim", "zn", "indus", "nox", "rm", "age",
           "dis", "tax", "ptratio", "medv"]
FEATURES = ["crim", "zn", "indus", "nox", "rm",
            "age", "dis", "tax", "ptratio"]
LABEL = "medv"

training_set = pd.read_csv("boston_train.csv", skipinitialspace=True,
                           skiprows=1, names=COLUMNS)
test_set = pd.read_csv("boston_test.csv", skipinitialspace=True,
                       skiprows=1, names=COLUMNS)
prediction_set = pd.read_csv("boston_predict.csv", skipinitialspace=True,
                             skiprows=1, names=COLUMNS)

In [4]:
feature_cols = [tf.contrib.layers.real_valued_column(k)
                  for k in FEATURES]

In [5]:
regressor = tf.contrib.learn.DNNRegressor(feature_columns=feature_cols,
                                          hidden_units=[10, 10],
                                          model_dir="/tmp/boston_model")

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x0000029EC4879B00>, '_master': '', '_is_chief': True, '_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_tf_random_seed': None, '_task_type': None, '_keep_checkpoint_max': 5, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_evaluation_master': '', '_save_checkpoints_steps': None, '_save_summary_steps': 100, '_keep_checkpoint_every_n_hours': 10000, '_environment': 'local'}


In [6]:
def input_fn(data_set):
    feature_cols = {k: tf.constant(data_set[k].values)
                  for k in FEATURES}
    labels = tf.constant(data_set[LABEL].values)
    return feature_cols, labels

In [7]:
regressor.fit(input_fn=lambda: input_fn(training_set), steps=5000)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 5001 into /tmp/boston_model\model.ckpt.
INFO:tensorflow:loss = 29.8164, step = 5001
INFO:tensorflow:global_step/sec: 1215.47
INFO:tensorflow:loss = 29.6964, step = 5101
INFO:tensorflow:global_step/sec: 1281.19
INFO:tensorflow:loss = 29.7209, step = 5201
INFO:tensorflow:global_step/sec: 1281.2
INFO:tensorflow:loss = 29.5771, step = 5301
INFO:tensorflow:global_step/sec: 1332.44
INFO:tensorflow:loss = 29.5169, step = 5401
INFO:tensorflow:global_step/sec: 1350.44
INFO:tensorflow:loss = 29.5401, step = 5501
INFO:tensorflow:global_step/sec: 1281.2
INFO:tensorflow:loss = 29.5264, step = 5

DNNRegressor(params={'feature_columns': (_RealValuedColumn(column_name='crim', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='zn', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='indus', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='nox', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='rm', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='age', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='dis', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='tax', dimension=1, default_value=None, dtype=tf.float32, normalizer=None), _RealValuedColumn(column_name='ptratio', dimension=1, default_value=None, dtype=tf.float32, normalizer=None)),

In [8]:
ev = regressor.evaluate(input_fn=lambda: input_fn(test_set), steps=1)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-05-08-01:06:41
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-05-08-01:06:41
INFO:tensorflow:Saving dict for global step 10000: global_step = 10000, loss = 13.6297


In [9]:
loss_score = ev["loss"]
print("Loss: {0:f}".format(loss_score))

Loss: 13.629739


In [10]:
y = regressor.predict(input_fn=lambda: input_fn(prediction_set))
# .predict() returns an iterator; convert to a list and print predictions
predictions = list(itertools.islice(y, 6))
print ("Predictions: {}".format(str(predictions)))

Predictions: [32.087582, 17.912066, 22.922274, 34.490669, 15.160352, 19.022882]
